### Figures ###
This notebook contains the script used to create the bulk of data/analysis figures in the manuscript, including:
* Figure 4 (New KDEs for DZ data)
* Figure 6 (Analysis of Paleozoic DZ data)
* Figure 7 (Composite KDEs by time)
* Figure 8 (Composite KDEs by tectonic domain)

In [ ]:
# Imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib
from matplotlib import rc
rc("pdf", fonttype=42)

from geoscripts.dz import dz


In [ ]:
# Load samples using geoscripts

samples_published = dz.load_all()

samples_new = dz.load_all('data_processing/dz_rel_age_filter/')

samples = samples_published + samples_new

In [ ]:
# Define plotting function for column of DZ analyses

def plot_dz_columns(smps,save=False,filename=None,mda=True,plot_mapped=False,jcarb=False,mpeaks=False,**kwargs):
    
    # Set up axes, title, limits, and plot KDE
    
    nplots = len(smps)
    height = nplots*1.5
    
    # If only 1 samples input
    if len(smps)==1:
        fig,axs = plt.subplots(nplots,dpi=300,figsize=(7.5,height))
        smp = smps[0]
        smp.kde(ax=axs,**kwargs)
        axs.set_title(smp.name)
        axs.set_xlim(50,4000)

        # Add MDA information to plot
        if mda:
            mda_text = 'YGC: ' + str(round(smp.mda,1)) + ' ± ' + str(round(smp.mda_err,1))
            axs.text(0.7,0.7,mda_text,transform=axs.transAxes)

            mla_data = pd.read_csv('data_processing/MLA_rel_age_filter/'+smp.original+'_mla.csv',index_col=0)
            smp.mla = mla_data.loc['t'].values[0]

            # For MLA error, convert to 2s and propagate systematic uncertainty
            mla_syst = (smp.syst_238/100)*smp.mla

            smp.mla_err = np.sqrt((mla_data.loc['s[t]'].values[0]*2)**2 + mla_syst**2)  # Convert 1s to 2s

            mla_text = 'MLA: ' + str(round(smp.mla,1)) + ' ± ' + str(round(smp.mla_err,1))
            axs[k].text(0.7,0.5,mla_text,transform=axs[k].transAxes)

            ysg_text = 'YSG: ' + str(round(smp.ysg,1)) + ' ± ' + str(round(smp.ysg_err,1))
            axs.text(0.7,0.3,ysg_text,transform=axs.transAxes)
    
    # If multiple samples input
    else:
        fig,axs = plt.subplots(nplots,dpi=300,figsize=(7.5,height),sharex=True)
        for k,smp in enumerate(smps):
            smp.kde(ax=axs[k],**kwargs)
            axs[k].set_title(smp.name)
            axs[k].set_xlim(50,4000)
            
            # Add MDA information to plot
            if mda:
                mda_text = 'YGC: ' + str(round(smp.mda,1)) + ' ± ' + str(round(smp.mda_err,1))
                axs[k].text(0.7,0.7,mda_text,transform=axs[k].transAxes)

                mla_data = pd.read_csv('data_processing/MLA_rel_age_filter/'+smp.original+'_mla.csv',index_col=0)
                smp.mla = mla_data.loc['t'].values[0]
                smp.mla_err = mla_data.loc['s[t]'].values[0]*2 # Convert 1s to 2s

                mla_text = 'MLA: ' + str(round(smp.mla,1)) + ' ± ' + str(round(smp.mla_err,1))
                axs[k].text(0.7,0.5,mla_text,transform=axs[k].transAxes)

                ysg_text = 'YSG: ' + str(round(smp.ysg,1)) + ' ± ' + str(round(smp.ysg_err,1))
                axs[k].text(0.7,0.3,ysg_text,transform=axs[k].transAxes)

            # Add mapped depositional age as a gray bar
            if plot_mapped:
                trans = axs[k].get_xaxis_transform()
                axs[k].plot([smp.mapped_age[0],smp.mapped_age[1]],[0.02,0.02],color='brown',linewidth=3,clip_on=False,transform=trans)
                axs[k].set_ylim(bottom=0)

            # Add Early Jurassic and Late Carboniferous boxes
            if jcarb:
                ejmin = 174.1
                ejmax = 201.3

                lcarbmin = 298.9
                lcarbmax = 323.2

                axs[k].axvline(ejmax,linestyle='--',color='navy',zorder=0)
                axs[k].axvline(ejmin,linestyle='--',color='navy',zorder=0)
                axs[k].axvline(lcarbmax,linestyle='--',color='purple',zorder=0)
                axs[k].axvline(lcarbmin,linestyle='--',color='purple',zorder=0)
            
            # Add lines for major peaks
            if mpeaks:
                #peaks = [170,240,300,450,2000,2600]
                spans = [(150,190),(220,260),(260,340),(410,490),(550,700),(1500,2300),(2400,2800)]
                colors = ['#EE7733', '#0077BB', '#EE3377','#009988', '#CC3311', '#33BBEE', '#BBBBBB']

                for n,span in enumerate(spans):
                    axs[k].axvspan(*span,alpha=0.3,color=colors[n],lw=0)
                
    plt.tight_layout()
    
    # Save the figure as a file
    if save == True:
        fig.savefig(filename)
    
    return

In [ ]:
# Assign mapped ages to samples using included CSV.

map_data = pd.read_csv('map_ages.csv')
age_cols = ['50K Min','50K Max','200K Min','200K Max','500k Min','500K Max','N Min','N Max']
ages = map_data[age_cols]
ages.index = map_data['Sample Number']

for smp in samples:
    if smp.name in ages.index:
        minmap = np.min(ages.loc[smp.name,:])
        maxmap = np.max(ages.loc[smp.name,:])
        smp.mapped_age = (minmap,maxmap)
        print(smp.name,smp.mapped_age)



In [ ]:
# Add shorthand name for newly-published samples

nickname_dic = {'CT15076':'KZ1','CT15082':'KZ2','CT15092':'KZ3','CT15127':'KZ4','CT15099':'KZ5','CT15113':'KZ6','AB0913':'RU1','CT130918-2A':'SV1',
'100211-9A':'SV2','CT130918-9A':'SV3','CT130919-5A':'SV4','100411-5':'SV5','CT130919-8A':'SV6','AB0926':'SV7','100211-1A':'SV8'}

for smp in samples:
    if smp.name in nickname_dic:
        smp.original = smp.name
        smp.name = nickname_dic[smp.name]

        # Purge numbers <1 that shouldn't be in the age column
        smp.bestage = smp.bestage[smp.bestage>1]

        print(smp.name,smp.original)

    else:
        smp.original=smp.name



In [ ]:
# Create sorted list of SV and KZ samples

kz_samples = [smp for smp in samples if 'KZ' in smp.name]
kz_samples.sort(key=lambda x: x.name, reverse=True)
sv_samples = [smp for smp in samples if 'SV' in smp.name]
sv_samples.sort(key=lambda x: x.name, reverse=True)

In [ ]:
# Specify new colors for Svaneti samples
for smp in sv_samples:
    if smp.name=='SV1':
        smp.color='red'
    elif smp.name in ['SV2','SV3']:
        smp.color='purple'
    elif smp.name in ['SV7']:
        smp.color='skyblue'

# Specify color Kazbegi samples
for smp in kz_samples:
    if smp.name=='KZ1':
        smp.color='skyblue'

In [ ]:
# Get single-grain MDAs for SV and KZ samples
os.makedirs('data_processing/isoplotr_oldsamples/',exist_ok=True)

#for smp in sv_samples:
    #age_errors = pd.concat([smp.bestage,smp.besterror],axis=1).dropna(how='any')
    #ages_sorted = age_errors.sort_values(by=['Best Age'],ignore_index=True)
    #smp.ysg = ages_sorted.iloc[0,0]

    # Factor in systematic error
    #ysg_syst = (smp.syst_238/100)*smp.ysg 
    #smp.ysg_err = np.sqrt(ages_sorted.iloc[0,1]**2 + ysg_syst**2)

#for smp in kz_samples:
    #age_errors = pd.concat([smp.bestage,smp.besterror],axis=1).dropna(how='any')
    #ages_sorted = age_errors.sort_values(by=['Best Age'],ignore_index=True)
    #smp.ysg = ages_sorted.iloc[0,0]

    # Factor in systematic error
    #ysg_syst = (smp.syst_238/100)*smp.ysg 
    #smp.ysg_err = np.sqrt(ages_sorted.iloc[0,1]**2 + ysg_syst**2)

# Redo MDAs for Pz samples from Cowgill16 and Vasey20.

syst_238 = {'NW-GC':0.9,'N1':1.0,'N2':0.8,'N3':0.8,'N5':0.9}

for smp in samples:
    if smp.name in ['NW-GC','N1','N2','N3','N5']:
        smp.calc_bestage(col_238='238/206 U-Pb Age',col_207='206/207 Pb Age',err_238='238 Error',err_207='206 Error',use_err=True,err_lev='1sig',
        disc_cutoff=20,disc_age_cutoff=600)
        smp.convert_1sigto2sig()
        smp.syst_238 = syst_238[smp.name]

        smp.age_238 = smp.agedata['238/206 U-Pb Age'][smp.bestage.index]
        smp.age_235 = smp.agedata['235/207 U-Pb Age'][smp.bestage.index]

        #smp.age_238.reset_index(inplace=True,drop=True)
        #smp.age_235.reset_index(inplace=True,drop=True)
        
        smp.calc_mda(systematic=True,filter235238=True,cutoffs235238=(80,110))
        smp.calc_ysg(systematic=True,filter235238=False,cutoffs235238=(80,110))

        print(smp.ysg)
        
        # Output 235/238 concordant grains for MLA
        conc238235 = np.round((smp.age_238/smp.age_235*100),0)
        accept = (conc238235>=80) & (conc238235<=110) 

        iso_output = pd.DataFrame([],columns=['tbest','err'])
        iso_output['tbest'] = smp.bestage[accept]
        
        # Convert 2s errors to 1s
        iso_output['err'] = smp.besterror[accept]/2

        iso_output.to_csv('data_processing/isoplotr_oldsamples/' + smp.name + '.csv',index=False)

In [ ]:
ticks = [50,100,150,200,300,500,1000,2000,3000]

# Plot KZ samples
plot_dz_columns(kz_samples,save=True,filename='kz_kdes.pdf',method=None,bw_adjust=0.2,ticks=ticks,plot_mapped=True,mpeaks=True)

In [ ]:
# Plot SV samples
plot_dz_columns(sv_samples,save=True,filename='sv_kdes.pdf',method=None,bw_adjust=0.2,ticks=ticks,plot_mapped=True,mpeaks=True)

In [ ]:
# Plot RU
ru = [smp for smp in samples if 'RU' in smp.name]

#for smp in ru:
    #age_errors = pd.concat([smp.bestage,smp.besterror],axis=1).dropna(how='any')    
    #ages_sorted = age_errors.sort_values(by=['Best Age'],ignore_index=True)
    #smp.ysg = ages_sorted.iloc[0,0]
    #smp.ysg_err = ages_sorted.iloc[0,1]

# SV8 included in plot to ensure sizing is correct
plot_dz_columns([sv_samples[0],ru[0]],save=True,filename='ru_kde.pdf',method=None,bw_adjust=0.2,ticks=ticks,plot_mapped=True,mpeaks=True)

In [ ]:
# Plot Devonian-Carboniferous samples (Pz)
pz1 = [smp for smp in sv_samples if smp.name=='SV1']
pz2 = [smp for smp in samples_published if smp.name in ['N1','N2','N3']]
pz = pz1+pz2

pz.sort(key=lambda x: np.min(x.bestage))

for smp in pz:
    smp.color='red'
plot_dz_columns(pz,save=True,filename='pz_kdes.pdf',method=None,mda=True,bw_adjust=0.2,ticks=ticks,plot_mapped=True,jcarb=True)



In [ ]:
# Plot P-Tr samples
tr1 = [smp for smp in sv_samples if smp.name in ['SV2','SV3']]
tr2 = [smp for smp in samples_published if smp.name in ['N5','NW-GC']]
tr = tr1+tr2

tr.sort(key=lambda x: np.min(x.bestage))

for smp in tr:
    smp.color='purple'

plot_dz_columns(tr,save=True,filename='tr_kdes.pdf',method=None,mda=True,bw_adjust=0.2,ticks=ticks,plot_mapped=True,jcarb=True)

In [ ]:
# Plot tectonic domains

# Get red and green colors
reds = matplotlib.cm.get_cmap('Reds')
greens = matplotlib.cm.get_cmap('Greens')

# Assign samples to groups
dizi_names = ['NW-GC','N5','Khelra','WGC-2','SV8','SV7']
jsed_names = ['Khopuri','SV2','SV3','SV4']
jvolc_names = ['SW-GC','WGC-3','SV6','SV5','CT130924-9A']
jkaz_names = ['K3','KZ1','KZ2','Tskhradzmula','CGC-1']
kkaz_names = ['KZ4','KZ3','KZ6','KZ5','CGC-2']

lc_names = ['LC-1','LC-2','LC-3','TC-1','Tovuz']

# Get DZ objects for each group
dizi_smps = [smp for smp in samples if smp.name in dizi_names]
jsed_smps = [smp for smp in samples if smp.name in jsed_names]
jvolc_smps = [smp for smp in samples if smp.name in jvolc_names]
jkaz_smps = [smp for smp in samples if smp.name in jkaz_names]
kkaz_smps = [smp for smp in samples if smp.name in kkaz_names]
lc_smps = [smp for smp in samples if smp.name in lc_names]

for smp in lc_smps:
    smp.color='green'
    smp.bestage = smp.bestage[smp.bestage>65]

lc_smps.sort(key=lambda x: x.name)

# Create composite KDE for each group
dizi = dz.composite(dizi_smps,name='North of Dizi Series')
jsed = dz.composite(jsed_smps,name='Triassic-Jurassic Sedimentary')
jvolc = dz.composite(jvolc_smps,name='Jurassic Volcaniclastic')
jkaz = dz.composite(jkaz_smps,name='Jurassic Thrust Sheet')
kkaz = dz.composite(kkaz_smps,name='Cretaceous Thrust Sheet')
lc = dz.composite(lc_smps,name='Lesser Caucasus')

# Assign colors
color_dict = {dizi:reds(0.9),jsed:reds(0.75),jkaz:reds(0.5),kkaz:reds(0.25),jvolc:greens(0.5),lc:greens(0.9)}

# Organize into columns
blocks1 = [dizi,jsed,jvolc]
blocks2 = [jkaz,kkaz,lc]
allblocks = blocks1+blocks2

for block in allblocks:
    block.color=color_dict[block]

# Plot
plot_dz_columns(blocks1,save=True,filename='sv_blocks.pdf',method=None,mda=False,bw_adjust=0.2,ticks=ticks,mpeaks=True)
plot_dz_columns(blocks2,save=True,filename='kz_blocks.pdf',method=None,mda=False,bw_adjust=0.2,ticks=ticks,mpeaks=True)
plot_dz_columns(lc_smps,save=True,filename='lc.pdf',method=None,mda=False,bw_adjust=0.2,ticks=ticks)


In [ ]:
# Plot composite KDEs through time

# Create composite KDEs for existing Devonian-Carboniferous and Permian-Triassic samples.
devcar = dz.composite(pz,name='Devonian-Carboniferous',color='red')
ptr = dz.composite(tr,name='Permian-Triassic',color='purple')

# Create Jurassic composite KDE
j_smps = [smp for smp in samples if smp.name in ['KZ1','KZ2','SV4','SV5','SV6','SV7','K3','SW-GC','CT130924-9A']]
j = dz.composite(j_smps,color='blue',name='Jurassic')

# Create Cretaceous composite KDE
k_smps = [smp for smp in samples if smp.name in ['SV8','KZ3','KZ4','KZ5','KZ6']]
k = dz.composite(k_smps,color='forestgreen',name='Cretaceous')

# Plot in a column
plot_dz_columns([devcar,ptr,j,k],save=True,filename='age_dep.pdf',mda=False,method=None,bw_adjust=0.2,mpeaks=True)

In [ ]:
# Plot YSG vs. mapped age

fig, ax = plt.subplots(1,dpi=300,figsize=(5,5))
ax.set_aspect('equal')
ax.plot(np.arange(0,1000,100),np.arange(0,1000,100),color='black')
ax.set_xlabel('Mapped Age (Ma)')
ax.set_ylabel('Detrital Zircon\nYoungest Single Grain Age (Ma)')
ax.set_xlim(0,500)
ax.set_ylim(0,500)

ejmin = 174.1
ejmax = 201.3

lcarbmin = 298.9
lcarbmax = 323.2

ax.axvline(ejmin,linestyle='--',color='navy',linewidth=0.5)
ax.axvline(ejmax,linestyle='--',color='navy',linewidth=0.5)
ax.axvline(lcarbmax,linestyle='--',color='purple',zorder=0,linewidth=0.5)
ax.axvline(lcarbmin,linestyle='--',color='purple',zorder=0,linewidth=0.5)

for smp in sv_samples+kz_samples+tr2+pz2+ru:
    if smp.bestage.count()>15:
        xmin = smp.mapped_age[0]
        xmax = smp.mapped_age[1]
        ymin = smp.ysg - smp.ysg_err
        ymax = smp.ysg + smp.ysg_err

        xavg = (xmax+xmin)/2
        yavg = (ymin+ymax)/2

        ej_grains = smp.bestage[(smp.bestage<ejmax)&(smp.bestage>ejmin)]
        lcarb_grains = smp.bestage[(smp.bestage<lcarbmax)&(smp.bestage>lcarbmin)]

        if (ej_grains.count()>0)&(lcarb_grains.count()>0):
            plot_color = 'navy'
        elif lcarb_grains.count()>0:
            plot_color='purple'
        else:
            plot_color='red'
        
        ax.plot([xmin,xmax],[ymin,ymin],color=plot_color)
        ax.plot([xmin,xmax],[ymax,ymax],color=plot_color)
        ax.plot([xmin,xmin],[ymin,ymax],color=plot_color)
        ax.plot([xmax,xmax],[ymin,ymax],color=plot_color)

        # Add annotations (clutters graph)
        #ax.annotate(smp.name,(xmin-30,ymax))

        fig.savefig('ysg_mapped.pdf')

In [ ]:
# Plot MLA vs. mapped age

fig, ax = plt.subplots(1,dpi=300,figsize=(5,5))
ax.set_aspect('equal')
ax.plot(np.arange(0,1000,100),np.arange(0,1000,100),color='black')
ax.set_xlabel('Mapped Age (Ma)')
ax.set_ylabel('Detrital Zircon\nMaximum Likelihood Age (Ma)')
ax.set_xlim(0,500)
ax.set_ylim(0,500)

ejmin = 174.1
ejmax = 201.3

lcarbmin = 298.9
lcarbmax = 323.2

ax.axvline(ejmin,linestyle='--',color='navy',linewidth=0.5)
ax.axvline(ejmax,linestyle='--',color='navy',linewidth=0.5)
ax.axvline(lcarbmax,linestyle='--',color='purple',zorder=0,linewidth=0.5)
ax.axvline(lcarbmin,linestyle='--',color='purple',zorder=0,linewidth=0.5)

for smp in sv_samples+kz_samples+tr2+pz2+ru:
    if smp.bestage.count()>15:
        xmin = smp.mapped_age[0]
        xmax = smp.mapped_age[1]
        ymin = smp.mla - smp.mla_err
        ymax = smp.mla + smp.mla_err

        xavg = (xmax+xmin)/2
        yavg = (ymin+ymax)/2

        ej_grains = smp.bestage[(smp.bestage<ejmax)&(smp.bestage>ejmin)]
        lcarb_grains = smp.bestage[(smp.bestage<lcarbmax)&(smp.bestage>lcarbmin)]

        if (ej_grains.count()>0)&(lcarb_grains.count()>0):
            plot_color = 'navy'
        elif lcarb_grains.count()>0:
            plot_color='purple'
        else:
            plot_color='red'
        
        ax.plot([xmin,xmax],[ymin,ymin],color=plot_color)
        ax.plot([xmin,xmax],[ymax,ymax],color=plot_color)
        ax.plot([xmin,xmin],[ymin,ymax],color=plot_color)
        ax.plot([xmax,xmax],[ymin,ymax],color=plot_color)

        # Add annotations (clutters graph)
        #ax.annotate(smp.name,(xmin-30,ymax))

        fig.savefig('mla_mapped.pdf')

In [ ]:
# Plot composite KDEs for EEC and Africa/Gondwana

eec_names = ['Wang_Volga','Wang_Don','Wang_Dnieper','Safanova_Volga','Safanova_Don']

eec_smps = [smp for smp in samples if smp.name in eec_names]

eec = dz.composite(eec_smps,name='EEC')

gond_names = ['Niger','Niger2','Nile','Nile2','Congo','Congo2','Zambezi','Zambezi2','Orange','Orange2']

gond_smps = [smp for smp in samples if smp.name in gond_names]

gond = dz.composite(gond_smps,name='Africa')

plot_dz_columns([eec,gond],save=True,filename='eecgond_kde.pdf',mda=False,method=None,bw_adjust=0.2,mpeaks=True)

